In [2]:
from collections import Counter, OrderedDict
from itertools import product
import matplotlib.pyplot as plt
from random import choices

import numpy as np
import string
import sys
import re
import math

# in this piece of code, I leave out a bunch of thing for you to fill up modify.
# The current code may run into a ZeroDivisionError. Thus, you need to add Laplace first.
'''
Todo: 
1. Laplace smoothing
2. Naive Bayes prediction
3. All the output.

'''


with open('script.txt', encoding='utf-8') as f:
    data = f.read()
# len(data)

data = data.lower()
data = data.translate(str.maketrans('', '', string.punctuation))
data = re.sub('[^a-z]+', ' ', data)
data = ' '.join(data.split(' '))

allchar = ' ' + string.ascii_lowercase

unigram = Counter(data)
unigram_prob = {ch: round((unigram[ch]) / (len(data)), 4) for ch in allchar}# A dictionary

uni_list = [unigram_prob[c] for c in allchar]

total_p = 0
for ch_p in uni_list:
    total_p = total_p + ch_p
offset = total_p - 1
uni_list[0] = uni_list[0]-offset


print(uni_list)
# open a (new) file to write
outF = open("unigram_probabilities.txt", "w")
for i in range(len(uni_list)):
    outF.write(str(uni_list[i]))
    if i != len(uni_list) - 1:
        outF.write(",")
outF.close()



1.0
[0.1942, 0.0695, 0.0119, 0.0306, 0.0322, 0.0872, 0.0129, 0.0181, 0.0448, 0.0523, 0.0087, 0.0194, 0.0309, 0.0229, 0.0564, 0.065, 0.0133, 0.0004, 0.0428, 0.0535, 0.0685, 0.0245, 0.0058, 0.0162, 0.0008, 0.0168, 0.0004]


In [3]:
def ngram(n):
    # all possible n-grams
    d = dict.fromkeys([''.join(i) for i in product(allchar, repeat=n)],0)
    # update counts
    d.update(Counter([''.join(j) for j in zip(*[data[i:] for i in range(n)])]))
    return d

bigram = ngram(2)  # c(ab)
bigram_prob = {c: round(bigram[c] / unigram[c[0]], 4) for c in bigram}  # p(b|a)

trigram = ngram(3)
trigram_prob_la = {c: round((trigram[c] + 1) / (bigram[c[:2]] + 27), 4) for c in trigram}  # p(c|ab)


# open a (new) file to write
outF = open("bigram_transition_probabilities.txt", "w")
counter = 1
for ch in bigram:
    if counter == 27:
        outF.write(str(round(bigram_prob[ch], 4)))
        outF.write('\n')
        counter = 1
        continue
    else:
        outF.write(str(round(bigram_prob[ch], 4)))
        outF.write(',')
    counter += 1
outF.close()



In [4]:
import math
from decimal import *

bigram_prob_la = {c: round((bigram[c] + 1) / (unigram[c[0]] + 27), 4) for c in bigram}  # p(b|a)


total_p = 0
counter = 1
for ch in bigram:
    if bigram_prob_la[ch] == 0.0000:
        bigram_prob_la[ch] = 0.0001
    total_p = total_p + bigram_prob_la[ch]
    if counter == 27:
        offset = 1 - total_p 
        bigram_prob_la[ch] = bigram_prob_la[ch] + offset
        total_p = 0
        counter = 1
        continue
    counter += 1
    
# open a (new) file to write
outF = open("bigram_transition_probabilities_la.txt", "w")
counter = 1
for ch in bigram:
    if counter == 27:
        outF.write(str(round(bigram_prob_la[ch], 4)))
        outF.write('\n')
        counter = 1
        continue
    else:
        outF.write(str(round(bigram_prob_la[ch], 4)))
        outF.write(',')
    counter += 1
outF.close()



In [5]:



def gen_bi(c):
    w = [bigram_prob_la[c + i] for i in allchar]
    return choices(allchar, weights=w)[0]
    

def gen_tri(ab):
    w_tri = [trigram_prob_la[ab + i] for i in allchar]
    return choices(allchar, weights=w_tri)[0]   


def gen_sen(c, num):
    res = c + gen_bi(c)
    for i in range(num - 2):
        if bigram[res[-2:]] == 0:
            t = gen_bi(res[-1])
        else:
            t = gen_tri(res[-2:])
        res += t
    return res


# open a (new) file to write
outF = open("26_sentences_bi_tri.txt", "w")
for c in allchar:
    if c == ' ':
        continue
    one_sentence = gen_sen(c, 1000)
    outF.write(one_sentence)
    if c != 'z':
        outF.write('\n')
outF.close()



In [6]:
with open('script_young.txt', encoding='utf-8') as f:
    young = f.read() 

dict2 = Counter(young)
likeli = [dict2[c] / len(young) for c in allchar]

# open a (new) file to write
outF = open("likelihood_probabilities_young.txt", "w")
for i in range(len(likeli)):
    outF.write(str(likeli[i]))
    if i != len(likeli) - 1:
        outF.write(",")
outF.close()

outF = open("posterior_probabilities.txt", "w")
for i in range(27):
    post_p = (0.4*likeli[i]) / (0.4*likeli[i] + 0.6*uni_list[i])
    outF.write(str(round(post_p, 4)))
    if i == 26:
        break
    outF.write(',')
outF.close()

post_young = [round((0.4*likeli[i]) / (0.4*likeli[i] + 0.6*uni_list[i]), 4) for i in range(27)]
post_liu = [1 - post_young[i] for i in range(27)]

print(post_young)
print(post_liu)


[0.2392, 0.0262, 0.4465, 0.1655, 0.087, 0.0151, 0.3385, 0.2846, 0.0317, 0.0333, 0.9106, 0.3401, 0.1061, 0.2166, 0.0364, 0.0337, 0.3448, 0.9972, 0.0503, 0.0441, 0.0144, 0.1732, 0.7184, 0.2453, 0.9913, 0.2841, 0.9977]
[0.7608, 0.9738, 0.5535, 0.8345, 0.913, 0.9849, 0.6615, 0.7154, 0.9683, 0.9667, 0.08940000000000003, 0.6598999999999999, 0.8939, 0.7834, 0.9636, 0.9663, 0.6552, 0.0028000000000000247, 0.9497, 0.9559, 0.9856, 0.8268, 0.28159999999999996, 0.7547, 0.008700000000000041, 0.7159, 0.0022999999999999687]


In [7]:
import math

file1 = open("26_sentences_bi_tri.txt", "r")
Lines = file1.readlines()
list_label = []
 
for line in Lines:
    line_s = line.strip()
    num_letter = Counter(line_s)
    sum_p_young = 0
    sum_p_liu = 0
    i = 0
    for ch in allchar:
        sum_p_young += num_letter[ch]*math.log10(post_young[i])
        sum_p_liu   += num_letter[ch]*math.log10(post_liu[i])
        i += 1
    if sum_p_liu >= sum_p_young:
        list_label.append(0)
    else:
        list_label.append(1)

outF = open("label.txt", "w")
for i in range(26):
    outF.write(str(list_label[i]))
    if i == 25:
        break
    outF.write(',')
outF.close()